In [1]:
!git clone https://github.com/muqtasid87/benchmark_vlm

Cloning into 'benchmark_vlm'...
remote: Enumerating objects: 2212, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 2212 (delta 1), reused 11 (delta 1), pack-reused 2192 (from 1)
Receiving objects: 100% (2212/2212), 133.63 MiB | 21.29 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [9]:
 !pip install transformers codecarbon bitsandbytes accelerate

In [1]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from transformers import BitsAndBytesConfig
from transformers.image_utils import load_image
import torch
import csv
import platform
from transformers import pipeline
import psutil
import numpy as np
from PIL import Image
import os
import shutil
import time
from codecarbon import EmissionsTracker
import requests
from transformers import AutoModelForCausalLM
from transformers import AutoProcessor

In [2]:
model_id = ["microsoft/Phi-3.5-vision-instruct"]

In [3]:
# Image directory
img_dir = 'benchmark_vlm/master'

### Inference Function

In [4]:
# model = AutoModelForCausalLM.from_pretrained(
#   model_id[0],
#   device_map="cuda",
#   trust_remote_code=True,
#   torch_dtype="auto",
#   _attn_implementation='eager'
# )

# # for best performance, use num_crops=4 for multi-frame, num_crops=16 for single-frame.
# processor = AutoProcessor.from_pretrained(model_id[0],
#   trust_remote_code=True,
#   num_crops=16
# )

model.safetensors.index.json:   0%|          | 0.00/68.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.35G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-vision-instruct:
- processing_phi3_v.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/opt/conda/lib/python3.11/site-packages/transformers/models/auto/image_processing_auto.py:524: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/442 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/9.52k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [25]:
# Function to run inference
def infer_image(image_path, prompt):
    with open(image_path, "rb") as f:
          input_image = Image.open(f).convert("RGB")

    
    messages = [
    {"role": "user", "content": f"<|image_1|>\n{prompt}"},
]

    prompt = processor.tokenizer.apply_chat_template(
      messages,
      tokenize=False,
      add_generation_prompt=True
    )
    
    inputs = processor(prompt, input_image, return_tensors="pt").to("cuda:0")

    generation_args = {
        "max_new_tokens": 6,
        "temperature": 0.1,
        "do_sample": True,
    }
    
    generate_ids = model.generate(**inputs,
    eos_token_id=processor.tokenizer.eos_token_id,
    **generation_args
  )


    # remove input tokens
    generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
    response = processor.batch_decode(generate_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False)[0]

    print(response.lower())

    return response.lower()
    
    
    

In [26]:
# Define vehicle classes for validation
vehicle_classes = {
    "Class 1 - Motorcycles": ["motorcycle", "motorbike", "bike", "scooter", "motor", "moped"],
    "Class 2 - Car, SUV, Station Wagons": ["car", "sedan", "hatchback", "suv", "mpv", "crossover", "station wagon", "wagon", 'coupe', 'convertible', 'sports car'],
    "Class 3 - 2 axle lorry, van, pickup": ["pickup", "van", "minivan"],
    "Class 4 - 2 axles, 5 or 6 wheels bus or truck": ["bus", "truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer"],
    "Class 5 - 3 axle truck, bus": ["bus", "truck"],
    "Class 6 - 4 axles": ["coach", "bus", "truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer"],
    "Class 7 - 5 or more axles": ["coach", "bus", "truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer"],
    "bus 2" : ["coach", "bus"],
    "bus 3" : ["coach", "bus"],
    "truck 2" : ["truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer"],
    "truck 3" : ["truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer"],
    "truck 4" : ["truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer"],
    "truck 5" : ["truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer"],
    "van" : ["van", "minivan", 'minibus'],
    "pickup" : ["pickup", "pickup truck", "camper"],
}

valid_vehicle_types = {type for sublist in vehicle_classes.values() for type in sublist}
vehicle_prompt = f"What type of vehicle is this? Choose one from: {', '.join(valid_vehicle_types)}. Answer in one word"

### Validation function

In [20]:
# Define vehicle classes for validation
vehicle_classes = {
    "Class 1 - Motorcycles": ["motorcycle", "motorbike", "bike", "scooter", "motor", "moped"],
    "Class 2 - Car, SUV, Station Wagons": ["car", "sedan", "hatchback", "suv", "mpv", "crossover", "station wagon", "wagon", 'coupe', 'convertible', 'sports car'],
    "Class 3 - 2 axle lorry, van, pickup": ["pickup", "van", "minivan"],
    "Class 4 - 2 axles, 5 or 6 wheels bus or truck": ["bus", "truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer"],
    "Class 5 - 3 axle truck, bus": ["bus", "truck"],
    "Class 6 - 4 axles": ["coach", "bus", "truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer"],
    "Class 7 - 5 or more axles": ["coach", "bus", "truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer"],
    "bus 2" : ["coach", "bus"],
    "bus 3" : ["coach", "bus"],
    "truck 2" : ["truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer"],
    "truck 3" : ["truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer"],
    "truck 4" : ["truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer"],
    "truck 5" : ["truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck', "tow truck", "dump trailer"],
    "van" : ["van", "minivan", 'minibus'],
    "pickup" : ["pickup", "pickup truck", "camper"],
}

valid_vehicle_types = {type for sublist in vehicle_classes.values() for type in sublist}

In [21]:
# Validate predictions

def validate_prediction(folder_name, prediction, axles=None):
    # Extract folder label
    for class_name, valid_labels in vehicle_classes.items():
        if folder_name.startswith(class_name):
            if axles:  # Use axle count for bus/truck
                if axles == 1 or 0:
                  return False
                elif axles == 2:
                    return "2" in folder_name[-1]
                elif axles == 3:
                    return "3" in folder_name[-1]
                elif axles == 4:
                    return "4" in folder_name[-1]
                elif axles >= 5:
                    return "5" in folder_name[-1]
            if isinstance(valid_labels, list):
                return prediction in valid_labels
            return prediction == valid_labels
    return False

# Main Loop

In [22]:
from tqdm import tqdm

In [23]:
def main_loop(wrong_predictions_dir, img_dir=img_dir, use_axles=False, valid_vehicle_types=valid_vehicle_types):
    total_files = sum(len(files) for _, _, files in os.walk(img_dir))
    progress_bar = tqdm(total=total_files, desc="Processing Images", unit="file")
    # Main processing loop
    start_time = time.time()
    tracker = EmissionsTracker()
    tracker.start()
    results = []
    for root, dirs, files in os.walk(img_dir):
        for file in files:
            if file.endswith((".jpg", ".png", ".jpeg")):
                image_path = os.path.join(root, file)
                folder_name = os.path.basename(os.path.dirname(image_path))
                progress_bar.set_description(f"Processing: {folder_name}")
                progress_bar.update(1)  # Increment the progress bar

                try:
                    # First inference: vehicle type
                    # vehicle_prompt = f"<image> answer en What type of vehicle is this? Answer in only one word."
                    vehicle_prompt = f"What type of vehicle is this? Choose one from: {', '.join(valid_vehicle_types)}. Answer in one word"
                    prediction = infer_image(image_path, vehicle_prompt)
                    print(f"Vehicle Prediction: {prediction}")

                    # Handle buses and trucks
                    axles = None
                    if prediction in ["bus", "truck", 'trailer', 'lorry', 'tractor', 'tanker', 'dump truck',  "tow truck", "dump trailer"]:
                        # Check if it's a pickup truck
                        if prediction == 'truck':
                            pickup_prompt = "Is this a pickup truck? Respond with only yes or no."
                            is_pickup = infer_image(image_path, pickup_prompt).strip().lower()
                            print(f"Is Pickup Truck: {is_pickup}")
                            if is_pickup == "yes":
                                prediction = "pickup truck"
                            elif use_axles:
                                # Predict axles for non-pickup trucks
                                axle_prompt = "How many axles does this vehicle have? Output a single integer value."
                                axle_prediction = infer_image(image_path, axle_prompt)
                                print(f"Axle Prediction: {axle_prediction}")
                    
                                try:
                                    axles = int(axle_prediction)
                                except ValueError:
                                    print(f"Invalid axle prediction: {axle_prediction}")
                                    axles = None  # Fallback if parsing fails
                        elif use_axles:

                            axle_prompt = "How many axles does this vehicle have? Output a single integer value"
                            axle_prediction = infer_image(image_path, axle_prompt)
                            print(f"Axle Prediction: {axle_prediction}")
    
                            try:
                                axles = int(axle_prediction)
                            except ValueError:
                                print(f"Invalid axle prediction: {axle_prediction}")

                    # Validate prediction
                    is_correct = validate_prediction(folder_name, prediction, axles)
                    print(f"Validation Result: {is_correct}")

                    # Store results
                    results.append({
                        "image": image_path,
                        "folder_label": folder_name,
                        "prediction": prediction,
                        "axles": axles,
                        "is_correct": is_correct
                    })

                    # Handle wrong predictions
                    if not is_correct:
                        # Create a folder for the wrong prediction
                        wrong_folder = os.path.join(wrong_predictions_dir, prediction, str(axles))
                        os.makedirs(wrong_folder, exist_ok=True)

                        # Copy the image to the folder
                        shutil.copy(image_path, os.path.join(wrong_folder, file))
                except Exception as e:
                    print(f"Error processing {image_path}: {e}")
                    continue
    
    progress_bar.close()
    emissions = tracker.stop()
    # Stop the timer
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Total processing time: {elapsed_time:.2f} seconds")
    print(f"CO2 emissions: {emissions:.6f} kg")
    return (results, elapsed_time, emissions)

In [24]:
# Save results to CSV
def export_results(results, model_id, elapsed_time, emissions):
# Save results to CSV
    output_csv = f'experiment_data/{models}/results.csv'
    with open(output_csv, mode="w", newline="") as file:
        writer = csv.DictWriter(file, fieldnames=["image", "folder_label", "prediction", "axles", "is_correct"])
        writer.writeheader()
        writer.writerows(results)

    # Metrics calculation
    correct_predictions = sum(1 for r in results if r["is_correct"])
    total_images = len(results)
    accuracy = correct_predictions / total_images * 100 if total_images > 0 else 0
    wrong_predictions = total_images - correct_predictions

    # Per-class metrics
    class_metrics = {}
    for result in results:
        label = result["folder_label"]
        correct = result["is_correct"]
        if label not in class_metrics:
            class_metrics[label] = {"correct": 0, "total": 0}
        class_metrics[label]["total"] += 1
        if correct:
            class_metrics[label]["correct"] += 1

    # Aggregate class metrics
    per_class_accuracy = {
        label: (data["correct"] / data["total"]) * 100
        for label, data in class_metrics.items()
    }
    per_class_error_rate = {
        label: 100 - acc for label, acc in per_class_accuracy.items()
    }

    # Collect system configuration
    system_config = {
        "Processor": platform.processor(),
        "CPU Count": psutil.cpu_count(logical=True),
        "Memory (GB)": round(psutil.virtual_memory().total / (1024 ** 3), 2),
        "Elapsed Time (seconds)": elapsed_time,
        "CO2 Emissions (kg)": round(emissions, 6)
    }

    # Metrics dictionarya
    metrics = {
        "Total Images": total_images,
        "Correct Predictions": correct_predictions,
        "Wrong Predictions": wrong_predictions,
        "Accuracy (%)": accuracy,
        "Error Rate (%)": 100 - accuracy
    }

    # Save metrics to CSV
    metrics_csv = f'experiment_data/{models}/metrics.csv'
    with open(metrics_csv, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Model", "Metric", "Value"])
        for key, value in metrics.items():
            writer.writerow([model_id, key, value])
        for label, acc in per_class_accuracy.items():
            writer.writerow([model_id, f"Per-Class Accuracy: {label}", acc])
            writer.writerow([model_id, f"Per-Class Error Rate: {label}", per_class_error_rate[label]])
        for key, value in system_config.items():
            writer.writerow([model_id, key, value])

    # Print final metrics
    print("\n--- Metrics ---")
    for key, value in metrics.items():
        print(f"{key}: {value}")
    for label, acc in per_class_accuracy.items():
        print(f"Class '{label}' - Accuracy: {acc:.2f}%, Error Rate: {per_class_error_rate[label]:.2f}%")
    for key, value in system_config.items():
        print(f"{key}: {value}")

In [ ]:
for models in model_id:
    print(models)
    model = AutoModelForCausalLM.from_pretrained(
    models,
    device_map="cuda",
    trust_remote_code=True,
    torch_dtype="auto",
    _attn_implementation='eager'
    )

# for best performance, use num_crops=4 for multi-frame, num_crops=16 for single-frame.
    processor = AutoProcessor.from_pretrained(models,
    trust_remote_code=True,
    num_crops=16
    )
        
    
    wrong_predictions_dir = f'experiment_data/{models}/wrong_predictions'  # Directory to store wrongly predicted images
    os.makedirs(wrong_predictions_dir, exist_ok=True)
    run = main_loop(wrong_predictions_dir = wrong_predictions_dir)
    export_results(run[0], models, run[1], run[2])
    
        

microsoft/Phi-3.5-vision-instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Processing Images:   0%|          | 0/932 [00:00<?, ?file/s][codecarbon INFO @ 14:45:20] [setup] RAM Tracking...
[codecarbon INFO @ 14:45:20] [setup] CPU Tracking...
[codecarbon WARNING @ 14:45:20] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon INFO @ 14:45:22] CPU Model on constant consumption mode: AMD EPYC 7513 32-Core Processor
[codecarbon INFO @ 14:45:22] [setup] GPU Tracking...
[codecarbon INFO @ 14:45:22] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 14:45:22] >>> Tracker's metadata:
[codecarbon INFO @ 14:45:22]   Platform system: Linux-5.15.0-125-generic-x86_64-with-glibc2.35
[codecarbon INFO @ 14:45:22]   Python version: 3.11.10
[codecarbon INFO @ 14:45:22]   CodeCarbon version: 2.8.2
[codecarbon INFO @ 14:45:22]   Available RAM : 503.471 GB
[codecarbon INFO @ 14:45:22]   CPU count: 128
[codecarbon INFO @ 14:45:22]   CPU model: AMD EPYC 7513 32-

motorcycle
Vehicle Prediction: motorcycle
Validation Result: True


Processing: Class 1 - Motorcycles:   0%|          | 3/932 [00:06<26:42,  1.72s/file]

...

Processing: truck 5:  97%|█████████▋| 903/932 [09:01<00:16,  1.77file/s]

lorry
Vehicle Prediction: lorry
Validation Result: True


Processing: truck 5:  97%|█████████▋| 904/932 [09:01<00:14,  1.90file/s]

dump tr
Vehicle Prediction: dump tr
Validation Result: False


Processing: truck 5:  97%|█████████▋| 905/932 [09:02<00:13,  2.00file/s]